In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import pickle

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    confusion_matrix
)



In [2]:
TEST_DATA_PATH = '../files/input/test_data.csv.zip'
TRAIN_DATA_PATH = '../files/input/train_data.csv.zip'
MODEL_PATH = '../files/models/model.pkl.gz'
METRICS_PATH = '../files/output/metrics.json'

In [3]:
test_data = pd.read_csv(
	TEST_DATA_PATH,
	index_col=False,
	compression='zip'
)

train_data = pd.read_csv(
	TRAIN_DATA_PATH,
	index_col=False,
	compression='zip'
)

In [4]:
test_data = test_data.rename(columns={"default payment next month": "default"})
train_data = train_data.rename(columns={"default payment next month": "default"})
test_data = test_data.drop(columns=["ID"])
train_data = train_data.drop(columns=["ID"])

In [5]:
train_data = train_data.loc[train_data["MARRIAGE"] != 0]
train_data = train_data.loc[train_data["EDUCATION"] != 0]

test_data = test_data.loc[test_data["MARRIAGE"] != 0]
test_data = test_data.loc[test_data["EDUCATION"] != 0]
test_data["EDUCATION"] = test_data["EDUCATION"].apply(lambda x: 4 if x > 4 else x)
train_data["EDUCATION"] = train_data["EDUCATION"].apply(lambda x: 4 if x > 4 else x)

In [6]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

x_train = train_data.drop(columns=["default"])
y_train = train_data["default"]

x_test = test_data.drop(columns=["default"])
y_test = test_data["default"]

In [7]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las demas variables al intervalo [0, 1].
# - Selecciona las K mejores caracteristicas.
# - Ajusta un modelo de regresion logistica.
#


categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]
numerical_features = list(set(x_train.columns) - set(categorical_features))

preprocessor = ColumnTransformer(
	transformers=[
		("num", MinMaxScaler(), numerical_features),
		("cat", OneHotEncoder(), categorical_features)
	],
	remainder="passthrough"
)

k_best = SelectKBest(f_classif)

model = LogisticRegression(random_state=42)

pipeline = Pipeline(
	steps=[
		("preprocessor", preprocessor),
		("k_best", k_best),
		("model", model)
	]
)


In [8]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', MinMaxScaler(),
                                                  ['BILL_AMT1', 'PAY_AMT2',
                                                   'PAY_AMT5', 'BILL_AMT4',
                                                   'PAY_AMT3', 'PAY_4', 'PAY_5',
                                                   'LIMIT_BAL', 'PAY_AMT1',
                                                   'PAY_6', 'PAY_3',
                                                   'BILL_AMT6', 'BILL_AMT5',
                                                   'PAY_0', 'AGE', 'PAY_AMT4',
                                                   'BILL_AMT3', 'PAY_2',
                                                   'PAY_AMT6', 'BILL_AMT2']),
                                                 ('cat', OneHotEncoder(),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE'])])),
                ('k_best', SelectKBest()),
                ('model', LogisticRegression(random_state=42))])

In [9]:
#
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
#

param_grid = {
	"k_best__k": range(1, len(x_train.columns) + 1),
	"model__C": np.logspace(-3, 3, 7)
}


grid_search = GridSearchCV(
	pipeline,
	param_grid=param_grid,
	cv=10,
	scoring="balanced_accuracy",
	n_jobs=-1,
	refit=True,
	verbose=2
)

grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 161 candidates, totalling 1610 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         ['BILL_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT5',
                                                                          'BILL_AMT4',
                                                                          'PAY_AMT3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'LIMIT_BAL',
                                                                          'PAY_AMT1',
                                                                          'PAY_6',
                                                                          'PAY_3',
                                                                          'BILL_AMT6',
                                                                          'BILL_AMT5',
                                                                          'PAY_0',
                                                                          'AGE',
                                                                          'PAY_AMT4',
                                                                          'BILL_AMT3',
                                                                          'PAY_2',
                                                                          'PAY_AMT6',
                                                                          'BILL_AMT2']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('k_best', SelectKBest()),
                                       ('model',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid={'k_best__k': range(1, 24),
                         'model__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
             scoring='balanced_accuracy', verbose=2)

In [11]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

with gzip.open(MODEL_PATH, 'wb') as f:
	pickle.dump(grid_search, f)

In [12]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#

metrics = {}

y_train_pred = grid_search.predict(x_train)
y_test_pred = grid_search.predict(x_test)

In [13]:
y_train_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(20953,))

In [15]:
metrics['train'] = {
    'type': 'metrics',
    'dataset': 'train',
	'precision': precision_score(y_train, y_train_pred, zero_division=0),
	'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
	'recall': recall_score(y_train, y_train_pred, zero_division=0),
	'f1_score': f1_score(y_train, y_train_pred, zero_division=0)
}

metrics['test'] = {
    'type': 'metrics',
    'dataset': 'test',
	'precision': precision_score(y_test, y_test_pred, zero_division=0),
	'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
	'recall': recall_score(y_test, y_test_pred, zero_division=0),
	'f1_score': f1_score(y_test, y_test_pred, zero_division=0)
}


# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}

cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

metrics['train_cm'] = {
    'type': 'cm_matrix',
	'dataset': 'train',
	'true_0': {"predicted_0": int(cm_train[0][0]), "predicted_1": int(cm_train[0][1])},
	'true_1': {"predicted_0": int(cm_train[1][0]), "predicted_1": int(cm_train[1][1])}
}

metrics['test_cm'] = {
    'type': 'cm_matrix',
	'dataset': 'test',
	'true_0': {"predicted_0": int(cm_test[0][0]), "predicted_1": int(cm_test[0][1])},
	'true_1': {"predicted_0": int(cm_test[1][0]), "predicted_1": int(cm_test[1][1])}
}


with open(METRICS_PATH, 'w') as f:
	f.write(json.dumps(metrics['train'])+'\n')
	f.write(json.dumps(metrics['test'])+'\n')
	f.write(json.dumps(metrics['train_cm'])+'\n')
	f.write(json.dumps(metrics['test_cm'])+'\n')
	
